1. Are these layers only on drive or also on GEE/Geoserver? 

2. Do style files remain same as the block level style files for Pan India datasets as well? 

3. All layers to be generated or only few? 

In [557]:
!pip install earthengine-api


Defaulting to user installation because normal site-packages is not writeable


In [55]:
import rasterio
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from shapely.geometry import Polygon
import requests
import xml.etree.ElementTree as ET
from io import BytesIO
import pystac
from pystac.extensions.table import TableExtension
from urllib.parse import urlparse, parse_qs
from pystac import Asset, MediaType
from pystac.extensions.classification import ClassificationExtension, Classification
from pystac.extensions.raster import RasterExtension,RasterBand
from pystac.extensions.projection import ProjectionExtension
import datetime
import ee
ee.Initialize(project='ee-corestackdev')

In [47]:


sheet_id = '1rSg8Zm0RHQ7wgyVr7ZZoDj66CB88Om7gbXfqLhDK8HI'

csv_export_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={gid}'


df = pd.read_csv(csv_export_url)

In [48]:
df

,SNo,Layer Name,GEE asset link,Raster/Vector File,File type and size,Style file url
0,1,Land Use Land Cover,https://code.earthengine.google.com/?asset=pro...,LULC_2017-2018,NaN,https://github.com/core-stack-org/QGIS-Styles/...


In [49]:
gee_links = df['GEE asset link']

In [50]:
gee_links

0    https://code.earthengine.google.com/?asset=pro...
Name: GEE asset link, dtype: object

In [52]:
url_string = gee_links.iloc[0]

In [53]:
url_string

'https://code.earthengine.google.com/?asset=projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018'

In [56]:
from urllib.parse import urlparse, parse_qs


In [57]:
parsed_url = urlparse(url_string)
query_params = parse_qs(parsed_url.query)

In [58]:
query_params

{'asset': ['projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018']}

In [60]:
asset_id = query_params['asset'][0]
print(asset_id)

projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018


In [61]:


##assets
#asset_id='projects/corestack-datasets/assets/datasets/Stream_Order_Raster_India'
#asset_id='projects/corestack-datasets/assets/datasets/WRI/LandscapeRestorationOpportunities'
#asset_id='projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018'

image = ee.Image(asset_id)
image_info = image.getInfo()

In [62]:
image_info

{'type': 'Image',
 'bands': [{'id': 'predicted_label',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [326117, 322830],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    68.11403776613543,
    0,
    -8.983152841195215e-05,
    37.0783226781469]}],
 'version': 1752832389901293,
 'id': 'projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018',
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[79.78650796613842, 37.078367773348994],
    [76.35344165455938, 37.07836776114237],
    [71.4327133847431, 37.07836773389202],
    [68.11396986781955, 37.07836766249718],
    [68.11399179206582, 8.076693122847033],
    [70.63166458964866, 8.075294360667307],
    [73.95029531922614, 8.073911283840776],
    [77.15449046492586, 8.075691767707651],
    [80.8164278315096, 8.07162220728194],
    [84.70723627834819, 8.074865085320496],
    [87.91143142740033, 8.074865080887003],
   

In [4]:
crs = image_info['bands'][0]['crs']
transform = image_info['bands'][0]['crs_transform']
width = image_info['bands'][0]['dimensions'][0]
height = image_info['bands'][0]['dimensions'][1]
bands = len(image_info['bands'])

In [5]:
min_x = transform[2]
max_y = transform[5]
max_x = min_x + width * transform[0]
min_y = max_y + height * transform[4]
bbox = [min_x, min_y, max_x, max_y]


In [6]:
bbox

[68.18625, 8.07791666666667, 97.41513888888888, 37.07819444444445]

In [7]:
print("CRS:", crs)
print("Bounds:", bbox)
print("Width x Height:", width, "x", height)
print("Bands:", bands)

CRS: EPSG:4326
Bounds: [68.18625, 8.07791666666667, 97.41513888888888, 37.07819444444445]
Width x Height: 105224 x 104401
Bands: 1


In [10]:

item_id = 'Stream order'

raster_item = pystac.Item(id=item_id,
                      geometry=10,
                      bbox=bbox,
                      datetime=datetime.datetime.now(datetime.timezone.utc),
                      properties={
                        #   title
                        # description
                          # "gsd": gsd, #adding this in raster extension 
                      })

In [11]:
proj_ext = ProjectionExtension.ext(raster_item, add_if_missing=True)
proj_ext.epsg = crs
#proj_ext.shape = [shape[0], shape[1]]

In [12]:
raster_item

<Item id=Stream order>

In [568]:
raster_item.add_asset("data", Asset(
    href=asset_id,
    media_type=MediaType.GEOTIFF,
    roles=["data"],
    title="Raster Layer"
))

In [569]:
raster_ext = RasterExtension.ext(raster_item.assets["data"], add_if_missing=True)
raster_band = RasterBand.create(
    #data_type=data_type, 
    spatial_resolution=10,
    # nodata=nodata
)
raster_ext.bands = [raster_band]

In [570]:
github_style_file='https://raw.githubusercontent.com/core-stack-org/QGIS-Styles/main/Restoration/Restoration_style.qml'

In [571]:
def parse_raster_style_file(github_style_file):

    try:
        response = requests.get(github_style_file, verify=False)
        response.raise_for_status()
        
    
        tree = ET.parse(BytesIO(response.content))
        root = tree.getroot()
        classes = []

        for entry in root.findall(".//paletteEntry"):
            class_info = {}
            for attr_key, attr_value in entry.attrib.items():
                if attr_key == "value":
                    try:
                        class_info[attr_key] = int(attr_value)
                    except ValueError:
                        class_info[attr_key] = attr_value
                else:
                    class_info[attr_key] = attr_value
            classes.append(class_info)

        if not classes:
            for entry in root.findall(".//item"):
                class_info = {}
                for attr_key, attr_value in entry.attrib.items():
                    if attr_key == "value":
                        try:
                            class_info[attr_key] = int(attr_value)
                        except ValueError:
                            class_info[attr_key] = attr_value
                    else:
                        class_info[attr_key] = attr_value
                classes.append(class_info)
        return classes

    except requests.exceptions.RequestException as e:
        print(f"Error fetching QML file from URL: {e}")
        return None
    except ET.ParseError as e:
        print(f"Error parsing XML from QML file: {e}")
        return None

In [572]:
style_info = parse_raster_style_file(github_style_file)

In [573]:
style_info

[{'color': '#d2d2d2', 'value': 0, 'label': 'Excluded', 'alpha': '0'},
 {'color': '#d79b0f',
  'value': 1,
  'label': 'Mosain Restoration',
  'alpha': '255'},
 {'color': '#0f077c',
  'value': 2,
  'label': 'Wide scale Restoration',
  'alpha': '255'},
 {'color': '#4fbc14', 'value': 3, 'label': 'Protection', 'alpha': '255'}]

In [574]:
classification_ext = ClassificationExtension.ext(raster_item.assets["data"], add_if_missing=True)
stac_classes = []
for cls in style_info:
    stac_class_obj = Classification.create(
        value=int(cls["value"]),
        name=cls.get("label") or f"Class {cls['value']}",
        description=cls.get("label"),
        color_hint=cls['color'].replace('#','')
    )
    stac_classes.append(stac_class_obj)
classification_ext.classes = stac_classes

In [575]:
raster_item.add_asset("style", Asset(
    href=github_style_file,
    media_type=MediaType.XML,
    roles=["metadata"],
    title="Raster Style (QML)"
))

In [576]:
raster_item

<Item id=Landscape-Restoration-WRI>

In [577]:
output_dir = '../data/'


raster_thumbnail_path = os.path.join(
    output_dir,
    asset_id.split('.')[0] + '_thumbnail.png'
)

In [578]:
# def generate_raster_thumbnail(asset_id,
#                               style_info,
#                               output_path
#                               ):
    
#     unique_raster_values = np.unique(asset_id.compressed() if isinstance(asset_id, np.ma.MaskedArray) else asset_id)
#     # Filter QML info to only include values present in the raster data
#     filtered_style_info = [cls for cls in style_info if cls.get('value') in unique_raster_values]
    
#     values = [cls['value'] for cls in filtered_style_info if 'value' in cls]
#     colors = [cls['color'] for cls in filtered_style_info if 'color' in cls]
    
#     # print(f"Parsed QML values: {values}")
#     # print(f"Parsed QML colors: {colors}")
        
#     try:
#         if not values or not colors or len(values) != len(colors):
#             raise ValueError("Invalid or insufficient palette information in QML file.")
    
#         sorted_indices = np.argsort(values)
#         sorted_values = np.array(values)[sorted_indices]
#         sorted_colors = np.array(colors)[sorted_indices]

#         cmap = ListedColormap(sorted_colors)
#         bounds = np.array(sorted_values) - 0.5
#         bounds = np.append(bounds, sorted_values[-1] + 0.5)
#         norm = Normalize(vmin=bounds.min(), vmax=bounds.max())

#     except ValueError as e:
#         print(f"Skipping palette generation due to error: {e}. Using a default colormap.")
#         cmap = 'gray'
#         norm = None
#     plt.figure(figsize=(3, 3), dpi=100)
    
#     plt.imshow(asset_id, cmap=cmap, norm=norm, interpolation='none')
#     plt.axis('off')

#     #os.makedirs(os.path.dirname(out_path), exist_ok=True)
#     plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

In [579]:
# os.makedirs(os.path.dirname(raster_thumbnail_path),exist_ok=True)

In [580]:
# generate_raster_thumbnail(
#     asset_id=asset_id,
#     style_info=style_info,
#     output_path= raster_thumbnail_path 
# )

In [581]:
root_catalog = pystac.Catalog(
    id="PANindia",
    title="STAC Catalog for PAN India",
    description="STAC items for PAN india."
)
root_catalog.add_item(raster_item)

print("STAC Item and Catalog created successfully!")
print("STAC Item ID:", raster_item.id)
print("Item's Parent Catalog:", raster_item.get_parent().id)

STAC Item and Catalog created successfully!
STAC Item ID: Landscape-Restoration-WRI
Item's Parent Catalog: PANindia


In [582]:
output_dir = '../data/'

root_catalog.normalize_hrefs(os.path.join(output_dir, 'catalog.json'))

root_catalog.save(
    dest_href=output_dir,
    catalog_type=pystac.CatalogType.SELF_CONTAINED
)

### For vector layer

In [19]:

#asset_id='projects/corestack-datasets/assets/datasets/Watershed_pan_india'

##6.21GB
asset_id='projects/corestack-datasets/assets/datasets/drainage-line/pan_india_drainage_lines'

##61KB
#asset_id='projects/corestack-datasets/assets/datasets/Reservoir_Stations_pan_india'


In [20]:
feature_collection = ee.FeatureCollection(asset_id)

first_feature_properties = feature_collection.first().toDictionary().getInfo()

column_schema = {}
    
for column_name, value in first_feature_properties.items():
    inferred_type = type(value).__name__
        

    column_schema[column_name] = inferred_type
        
print("Column Names and Inferred Data Types:")
for column, dtype in column_schema.items():
        print(f"  - {column}: {dtype}")

    



Column Names and Inferred Data Types:
  - ORDER: int
  - cat: int
  - end_node: str
  - id: int
  - network: int
  - start_node: str
  - stream_typ: str
  - type_code: int


In [21]:
collection_geometry = feature_collection.geometry()

bbox= collection_geometry.bounds().getInfo()

print(bbox)

EEException: User memory limit exceeded.

In [590]:
item_id_vector='Reservoir_Stations'

vector_item = pystac.Item(
    id=item_id_vector,
    geometry=10,
    bbox=bbox,
    datetime=datetime.datetime.now(datetime.timezone.utc),
    properties={
        # "title": title,
        # "description": f"Vector data for {os.path.splitext(vector_filename)[0]} in {block} of {state_title}",
        # "start_datetime": start_date.isoformat() + 'Z',
        # "end_datetime": end_date.isoformat() + 'Z',
    }
)

In [ ]:
vector_item

<Item id=Drainage lines>

In [ ]:
vector_gdf_dtypes = pd.DataFrame(column_schema.items(), columns=['column_name', 'column_dtype'])


In [ ]:
vector_desc_df = pd.read_csv('../data/column_descriptions.csv')

In [ ]:
vector_desc_df.rename(columns={'column_name_description' :  'column_description'},inplace=True)

In [ ]:
vector_desc_df['layer_name'].unique()

array(['admin_boundary', 'aquifer_vector', 'drainage_lines', 'swb3',
       'swb2', 'change_vector_Afforestation',
       'change_vector_CropIntensity', 'change_vector_Deforestation',
       'change_vector_Degradation', 'change_vector_Urbanization',
       'cropping_intensity_2017-23', 'drought_2017_2022',
       'restoration_vector', 'soge_vector', 'lulc_vector',
       'lulcXplains_clusters', 'terrain_clusters',
       'tree_health_ccd_vector_2017', 'tree_health_overall_change_vector',
       'well_depth_annual', 'well_depth_net_value'], dtype=object)

In [ ]:
vector_layername='drainage_lines'

In [ ]:
vector_desc_filtered_df = vector_desc_df[vector_desc_df['layer_name'] == vector_layername.replace('_vector', '')]


In [ ]:
vector_merged_df = vector_gdf_dtypes.merge(
    vector_desc_filtered_df[['column_name', 'column_description']],
    on='column_name',
    how='left'
).fillna('')

In [ ]:
vector_merged_df

,column_name,column_dtype,column_description
0,ORDER,int,Strahler order of stream within a network
1,cat,int,category for drainage feature
2,end_node,str,unique identifier for the downstream point of ...
3,id,int,id
4,network,int,
5,start_node,str,unique identifier for the upstream point of dr...
6,stream_typ,str,\nstream type: start or intermediate
7,type_code,int,code to classify the type of drainage feature:...


In [23]:
table_ext = TableExtension.ext(vector_item, add_if_missing=True)

NameError: name 'vector_item' is not defined

In [ ]:
table_ext.columns = [
    {
        "name": row['column_name'],
        "type": str(row['column_dtype']),
        "description" : row['column_description']
    }
    for ind,row in vector_merged_df.iterrows()
]

In [ ]:
vector_item.add_asset("data", Asset(
    href='https://code.earthengine.google.com/?asset=projects/corestack-datasets/assets/datasets/drainage-line/pan_india_drainage_lines',
    media_type='GEOJSON',
    roles=["data"],
    title="Vector Layer"
))

In [ ]:
vector_item.add_asset("style", Asset(
    href='https://github.com/core-stack-org/QGIS-Styles/blob/main/Hydrology/Drainage-Layer-Style.qml',
    media_type=MediaType.XML,
    roles=["metadata"],
    title="Vector Style (QML)"
))

In [ ]:
root_catalog = pystac.Catalog(
    id="PANindia_vector",
    title="STAC Catalog for PAN India vector",
    description="STAC items for PAN india."
)
root_catalog.add_item(vector_item)

print("STAC Item and Catalog created successfully!")
print("STAC Item ID:", vector_item.id)
print("Item's Parent Catalog:", vector_item.get_parent().id)

STAC Item and Catalog created successfully!
STAC Item ID: Drainage lines
Item's Parent Catalog: PANindia_vector


In [ ]:
output_dir = '../data/'

root_catalog.normalize_hrefs(os.path.join(output_dir, 'catalog.json'))

root_catalog.save(
    dest_href=output_dir,
    catalog_type=pystac.CatalogType.SELF_CONTAINED
)